In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
import numpy as np
from sklearn.cluster import KMeans

def stratified_cluster_split(X, y, test_size=0.5, random_state=42):
    X_sub_list = []
    y_sub_list = []
    X_rest_list = []
    y_rest_list = []

    classes = np.unique(y)

    rng = np.random.RandomState(seed=random_state)

    for c in classes:
        X_c = X[y == c]
        y_c = y[y == c]

        kmeans = KMeans(n_clusters=2, random_state=random_state)
        cluster_labels = kmeans.fit_predict(X_c)

        cluster_0_idx = np.where(cluster_labels == 0)[0]
        cluster_1_idx = np.where(cluster_labels == 1)[0]

        if len(cluster_0_idx) <= len(cluster_1_idx):
            base_idx = cluster_0_idx
        else:
            base_idx = cluster_1_idx

        n_select = int(len(X_c) * test_size)
        if n_select == 0:
            continue

        if len(base_idx) >= n_select:
            selected_idx = rng.choice(base_idx, size=n_select, replace=False)
        else:
            extra_needed = n_select - len(base_idx)
            other_idx = np.setdiff1d(np.arange(len(X_c)), base_idx)
            extra_idx = rng.choice(other_idx, size=extra_needed, replace=False)
            selected_idx = np.concatenate([base_idx, extra_idx])

        rest_idx = np.setdiff1d(np.arange(len(X_c)), selected_idx)

        X_sub_list.append(X_c[selected_idx])
        y_sub_list.append(y_c[selected_idx])
        X_rest_list.append(X_c[rest_idx])
        y_rest_list.append(y_c[rest_idx])

    X_sub = np.concatenate(X_sub_list, axis=0)
    y_sub = np.concatenate(y_sub_list, axis=0)
    X_rest = np.concatenate(X_rest_list, axis=0)
    y_rest = np.concatenate(y_rest_list, axis=0)

    return X_rest, X_sub, y_rest, y_sub


_, X_1, _, y_1 = stratified_cluster_split(x_test_adv_0_1, y_label_0_1, test_size=0.01, random_state=42)


In [6]:
X_1.shape

(10253, 56)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb1 = XGBClassifier()
xgb1.fit(X_1, y_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier()
rf1.fit(X_1, y_1)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt1 = DecisionTreeClassifier()
dt1.fit(X_1, y_1)

xgb
RF
DT


DecisionTreeClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Cluster/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Cluster/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  5  8 10] [16014  1266  8978    73 45748 41887     5]
(16014, 56) (16014,)
Save 0 to baseline_Def1.npy
(1266, 56) (1266,)
Save 1 to baseline_Def2.npy
(8978, 56) (8978,)
Save 2 to baseline_Def3.npy
(73, 56) (73,)
Save 3 to baseline_Def4.npy
(45748, 56) (45748,)
Save 5 to baseline_Def6.npy
(41887, 56) (41887,)
Save 8 to baseline_Def9.npy
(5, 56) (5,)
Save 10 to baseline_Def11.npy
Execution Time: 0.808350 seconds
(341913,)
[ 0  1  2  3  4  5  6  8 10] [157040     88   5523    206   2185 156689     26  20150      6]
(157040, 56) (157040,)
Save 0 to BIM_Def1.npy
(88, 56) (88,)
Save 1 to BIM_Def2.npy
(5523, 56) (5523,)
Save 2 to BIM_Def3.npy
(206, 56) (206,)
Save 3 to BIM_Def4.npy
(2185, 56) (2185,)
Save 4 to BIM_Def5.npy
(156689, 56) (156689,)
Save 5 to BIM_Def6.npy
(26, 56) (26,)
Save 6 to BIM_Def7.npy
(20150, 56) (20150,)
Save 8 to BIM_Def9.npy
(6, 56) (6,)
Save 10 to BIM_Def11.npy
Execution Time: 0.824281 seconds
(341913,)
[0 1 2 3 4 5 6 8] [157577      5   5090   

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Cluster/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Cluster/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  5  8 10] [14645   418  6992    35 47329 44548     4]
(14645, 56) (14645,)
Save 0 to baseline_Def1.npy
(418, 56) (418,)
Save 1 to baseline_Def2.npy
(6992, 56) (6992,)
Save 2 to baseline_Def3.npy
(35, 56) (35,)
Save 3 to baseline_Def4.npy
(47329, 56) (47329,)
Save 5 to baseline_Def6.npy
(44548, 56) (44548,)
Save 8 to baseline_Def9.npy
(4, 56) (4,)
Save 10 to baseline_Def11.npy
Execution Time: 0.918512 seconds
(341913,)
[0 2 3 4 5 8] [168796   2687    381     33 161663   8353]
(168796, 56) (168796,)
Save 0 to BIM_Def1.npy
(2687, 56) (2687,)
Save 2 to BIM_Def3.npy
(381, 56) (381,)
Save 3 to BIM_Def4.npy
(33, 56) (33,)
Save 4 to BIM_Def5.npy
(161663, 56) (161663,)
Save 5 to BIM_Def6.npy
(8353, 56) (8353,)
Save 8 to BIM_Def9.npy
Execution Time: 2.012444 seconds
(341913,)
[0 2 3 4 5 8] [171087   2834    260    502 160899   6331]
(171087, 56) (171087,)
Save 0 to FGSM_Def1.npy
(2834, 56) (2834,)
Save 2 to FGSM_Def3.npy
(260, 56) (260,)
Save 3 to FGSM_Def4.npy
(502, 56) (

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt1.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Cluster/UNSW_Input1/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Cluster/UNSW_Input1/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  5  8  9 10] [16202  1281 17111   201 35399 42864   294   619]
(16202, 56) (16202,)
Save 0 to baseline_Def1.npy
(1281, 56) (1281,)
Save 1 to baseline_Def2.npy
(17111, 56) (17111,)
Save 2 to baseline_Def3.npy
(201, 56) (201,)
Save 3 to baseline_Def4.npy
(35399, 56) (35399,)
Save 5 to baseline_Def6.npy
(42864, 56) (42864,)
Save 8 to baseline_Def9.npy
(294, 56) (294,)
Save 9 to baseline_Def10.npy
(619, 56) (619,)
Save 10 to baseline_Def11.npy
Execution Time: 0.094957 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [130565   6114  49829   2856   3333  98251    343      5  43462   6821
    334]
(130565, 56) (130565,)
Save 0 to BIM_Def1.npy
(6114, 56) (6114,)
Save 1 to BIM_Def2.npy
(49829, 56) (49829,)
Save 2 to BIM_Def3.npy
(2856, 56) (2856,)
Save 3 to BIM_Def4.npy
(3333, 56) (3333,)
Save 4 to BIM_Def5.npy
(98251, 56) (98251,)
Save 5 to BIM_Def6.npy
(343, 56) (343,)
Save 6 to BIM_Def7.npy
(5, 56) (5,)
Save 7 to BIM_Def8.npy
(43462, 56) (43462,)
Save 8 to BIM_Def9

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}1")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_Cluster/UNSW_Input1/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input1_Cluster.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB1,16014,1266,8978,73,0,45748,0,0,41887,0,5,113971
1,XGB1,157040,88,5523,206,2185,156689,26,0,20150,0,6,341913
2,XGB1,157577,5,5090,206,360,158834,54,0,19787,0,0,341913
3,XGB1,157040,88,5523,206,2185,156689,26,0,20150,0,6,341913
4,XGB1,174084,17,3296,215,387,156640,215,3178,3553,328,0,341913
5,XGB1,149685,385,7238,349,1779,154832,54,0,27582,0,9,341913
6,XGB1,52185,3773,24769,225,0,137690,0,0,123256,0,15,341913
7,XGB1,150656,1170,7502,25,105,118135,0,0,32397,27481,4442,341913
8,XGB1,155042,1056,4799,665,418,157360,75,0,22498,0,0,341913
9,XGB1,155381,81,5512,1113,1545,157509,4,0,20768,0,0,341913
